In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import yaml
from pathlib import Path

### Vehicle statistics
This notebook generates information on the investigated vehicles and vehicle types for the model

In [2]:
# Inputs and outputs
resources_path = Path("../resources")
results_path = Path("../results")
material_apth = Path("material")

figures_path = Path("figures")
tables_path = Path("tables")

In [3]:
# Load source information
df_sources = pd.read_excel(material_apth / "Vehicles - 2024-05-02.ods", sheet_name = "Sources")
df_sources

,Source,Link
0,a,https://www.free2move-lease.fr/nos-offres/nos-...
1,b,https://professionnels.renault.fr/offres-vehic...
2,c,https://professionnels.renault.fr/offres-vehic...


In [4]:
# Load vehicle information
df_vehicles = pd.read_excel(material_apth / "Vehicles - 2024-05-02.ods", sheet_name = "Vehicles")
df_vehicles.head()

,Source,Brand,Type,Monthly rent [EUR],First month rent [EUR],Duration [months],Average monthly rent [EUR],Volume [m3],Engine,Fuel consumption [L/100km],Emissions [gCO2eq/km],Electricity consumption [Wh/km],Consumption calculated,Battery [kWh],Battery calculated,Range [km]
0,a,Citroen,Jumpy,366,466,48,368.083333,3.7,T,6.2,162.0,NaN,NaN,NaN,NaN,NaN
1,a,Citroen,E-Berlingo,301,1036,48,316.312500,3.3,E,NaN,NaN,187.0,NaN,51.238,x,274.0
2,a,Citroen,Jumper,412,512,48,414.083333,11.5,T,7.8,206.0,NaN,NaN,NaN,NaN,NaN
3,a,Peugeot,Boxer,899,4005,48,963.708333,11.0,E,NaN,NaN,361.0,NaN,42.237,x,117.0
4,a,Peugeot,E-Expert,329,4510,48,416.104167,5.3,E,NaN,NaN,230.0,NaN,54.740,x,238.0


In [5]:
# Prepare table for annex
df_table = df_vehicles.copy()

column = "Source"
df_table[column] = df_table[column].apply(lambda x: "({})".format(x))

column = "Monthly rent [EUR]"
df_table[column] = df_table[column].round(0).astype(int).astype(str)

column = "First month rent [EUR]"
df_table[column] = df_table[column].round(0).astype(int).apply("{:,}".format).astype(str)

column = "Duration [months]"
df_table[column] = df_table[column].round(0).astype(int).astype(str)

column = "Average monthly rent [EUR]"
df_table[column] = df_table[column].round(0).astype(int).astype(str)

column = "Volume [m3]"
df_table["volume_sort"] = df_table[column].copy()
df_table[column] = df_table[column].round(1).astype(str)

column = "Engine"
df_table[column] = df_table[column].replace({ "T": "A", "E": "B" }) # for sorting

column = "Fuel consumption [L/100km]"
df_table[column] = df_table[column].round(1).fillna("-").astype(str)

column = "Emissions [gCO2eq/km]"
df_table[column] = df_table[column].fillna(-1).astype(int).astype(str).replace({ "-1": "-" })

column = "Electricity consumption [Wh/km]"
df_table[column] = df_table[column].fillna(-1).astype(int).astype(str).replace({ "-1": "-" })

column = "Consumption calculated"
df_table[column] = df_table[column] == "x" # for later

column = "Battery [kWh]"
df_table[column] = df_table[column].apply("{:.2f}".format).replace({ "nan": "-" })

column = "Battery calculated"
df_table[column] = df_table[column] == "x" # for later

column = "Range [km]"
df_table[column] = df_table[column].fillna(-1).astype(int).astype(str).replace({ "-1": "-" })

In [6]:
df_table = df_table.sort_values(by = [
    "Engine", "volume_sort", "Brand"
])

df_table = df_table.drop(columns = ["Engine", "volume_sort"])

In [7]:
f = df_table["Battery calculated"]
df_table.loc[f, "Battery [kWh]"] += "*"
df_table = df_table.drop(columns = ["Battery calculated"])

f = df_table["Consumption calculated"]
df_table.loc[f, "Electricity consumption [Wh/km]"] += "**"
df_table = df_table.drop(columns = ["Consumption calculated"])

In [8]:
units = []
updated_columns = []

for column in df_table.columns:
    if "[" in column:
        units.append(column.split("[")[1].split("]")[0].strip())
        updated_columns.append(column.split("[")[0].strip())
    else:
        units.append("")
        updated_columns.append(column)

df_table.columns = updated_columns

In [9]:
latex = df_table.to_latex(
    index = False, label = "tab:appendix_vehicles", caption = "Collected data on long-duration contracts and vehicle types"
)

latex = latex.replace("\\begin{table}", "\\begin{landscape}")
latex = latex.replace("\\end{table}", "\\end{landscape}")
latex = latex.replace("\\caption", "\\captionof{table}")

for column in df_table.columns:
    update = "\\makecell[c]{{{}}}".format(column.replace(" ", " \\\\ "))
    latex = latex.replace(column, update)

latex = latex.replace("\\bottomrule", """
    \\bottomrule
    \\multicolumn{10}{l}{\\footnotesize *Not indicated. Derived by multiplying consumption and range.} \\\\
    \\multicolumn{10}{l}{\\footnotesize **Not indicated. Derived by dividing battery capacity by range.} \\\\
    \\multicolumn{10}{l}{\\footnotesize Sources accessed in February 2020:} \\\\
""" + "\n".join([
    "\\multicolumn{{10}}{{l}}{{\\footnotesize ({}) \\url{{{}}}}} \\\\".format(row["Source"], row["Link"])
    for index, row in df_sources.iterrows()
]))

latex = latex.replace("consumption", "cons.")
latex = latex.replace("Electricity", "Elec.")
latex = latex.replace("\\midrule", "\\midrule " + " & ".join(["\\footnotesize [{}]".format(u) if len(u) > 0 else "" for u in units]) + "\\\\ \\midrule")
latex = latex.replace("lllllllllllll", "lll|rrrr|r|rr|rrr")

In [10]:
# Write annex table
with open(tables_path / "appendix_vehicles.tex", "w+") as f:
    f.write(latex)

In [11]:
# Plot costs by engine type
df_plot = df_vehicles.copy()
df_plot["Type"] = df_plot["Engine"]
df_plot["Type"] = df_plot["Type"].replace({ "T": "ICV", "E": "BEV" })

scatter_plot = px.scatter(
    df_plot, x = "Volume [m3]", y = "Average monthly rent [EUR]", color = "Type"
)

df_electric = pd.DataFrame({
    "Volume [m3]": np.linspace(0, 12),
    "Average monthly rent [EUR]": 0.0 + np.linspace(0, 12) * 80
})
df_electric["Type"] = "BEV"

df_thermic = pd.DataFrame({
    "Volume [m3]": np.linspace(0, 12),
    "Average monthly rent [EUR]": 180.0 + np.linspace(0, 12) * 15
})
df_thermic["Type"] = "ICV"

df_line = pd.concat([df_thermic, df_electric])
line_plot = px.line(df_line, x = "Volume [m3]", y = "Average monthly rent [EUR]", color = "Type")

figure = go.Figure(data = scatter_plot.data + line_plot.data)

figure.update_traces(selector = 0, showlegend = False)
figure.update_traces(selector = 1, showlegend = False)

figure.update_layout(
    width = 600,
    height = 200,
    margin=dict(
        l = 0,
        r = 0,
        b = 0,
        t = 0,
        pad = 0
    ),
    xaxis_title = "Volume [m3]",
    yaxis_title = "Monthly rent [EUR]"
)

figure.write_image(figures_path / "costs_by_engine.pdf")
figure

In [12]:
# Preapre table for model vehicle types
with open(resources_path / "vehicle_types.yml") as f:
    data = yaml.load(f, yaml.FullLoader)["vehicle_types"]

for vt in data.values():
    for key in vt["consumption"].keys():
        vt[key] = vt["consumption"][key]

    del vt["consumption"]
    
    if "externalities" in vt:
        for key in vt["externalities"].keys():
            vt[key] = vt["externalities"][key]

        del vt["externalities"]

df_stereo = pd.DataFrame.from_dict(data).T
df_stereo["electric"] = df_stereo["electric"].fillna(False).astype(bool)

fuel_cost_EUR_L = 1.8
df_stereo["distance_cost_EUR_100km"] = df_stereo["fuel_L_per_100km"].fillna(0.0) * fuel_cost_EUR_L

electricity_cost_EUR_Wh = 0.28 * 1e-3
df_stereo["distance_cost_EUR_100km"] += 1e2 * df_stereo["electricity_Wh_per_km"].fillna(0.0) * electricity_cost_EUR_Wh

electricity_emissions_gCO2eq_kWh = 90
f = df_stereo["co2eq_g_per_km"].isna()
df_stereo.loc[f, "co2eq_g_per_km"] = df_stereo.loc[f, "electricity_Wh_per_km"] * 1e-3 * electricity_emissions_gCO2eq_kWh

df_stereo["capacity"] = df_stereo["capacity"].astype(int).astype(str)
df_stereo["cost_EUR_per_month"] = df_stereo["cost_EUR_per_month"].astype(int).astype(str)
df_stereo["fuel_L_per_100km"] = df_stereo["fuel_L_per_100km"].fillna(-1).astype(int).astype(str).replace({ "-1": "-" })
df_stereo["electricity_Wh_per_km"] = df_stereo["electricity_Wh_per_km"].fillna(-1).astype(int).astype(str).replace({ "-1": "-" })
df_stereo["maximum_distance_km"] = df_stereo["maximum_distance_km"].fillna(-1).astype(int).astype(str).replace({ "-1": "-" })
df_stereo["co2eq_g_per_km"] = df_stereo["co2eq_g_per_km"].apply("{:.2f}".format).astype(str)
df_stereo["distance_cost_EUR_100km"] = df_stereo["distance_cost_EUR_100km"].apply("{:.2f}".format).astype(str)

df_stereo.loc[df_stereo["electric"], "co2eq_g_per_km"] += "*"
df_stereo.loc[~df_stereo["electric"], "distance_cost_EUR_100km"] += "**"
df_stereo.loc[df_stereo["electric"], "distance_cost_EUR_100km"] += "***"

df_stereo = df_stereo.T

df_stereo = df_stereo.loc[[
    "capacity", "cost_EUR_per_month", "fuel_L_per_100km", "electricity_Wh_per_km",
    "maximum_distance_km", "co2eq_g_per_km", "distance_cost_EUR_100km", "electric"
]]

df_stereo.columns = [
    c.replace("small_", "Small ").replace("medium_", "Medium ").replace("large_", "Large ").replace("thermic", "ICV").replace("electric", "BEV")
    for c in df_stereo.columns]

df_stereo = df_stereo[df_stereo.index != "electric"]

names = {
    "capacity": "Capacity",
    "cost_EUR_per_month": "Monthly cost [EUR]",
    "fuel_L_per_100km": "Fuel cons. [L/100km]",
    "electricity_Wh_per_km": "Elec. cons. [Wh/km]",
    "maximum_distance_km": "Range [km]",
    "co2eq_g_per_km": "Emissions [gCO2eq/km]",
    "distance_cost_EUR_100km": "Fuel cost [EUR/100km]",
}

df_stereo.index = [names[item] if item in names else item for item in df_stereo.index]

df_stereo.columns = ["\\makecell{{{}}}".format(c.replace(" ", "\\\\")) for c in df_stereo.columns]

latex = df_stereo.to_latex(
    label = "tab:stereo_vehicles", caption = "Vehicle types used in the simulation"
)

latex = latex.replace("\\begin{table}", "\\begin{table}\n\\centering")
latex = latex.replace("Emissions", "\\midrule\nEmissions")
latex = latex.replace("Fuel", "\\midrule\nFuel")

latex = latex.replace("\\bottomrule", """
    \\bottomrule
    \\multicolumn{7}{l}{\\footnotesize *Based on 90gCO2eq/kWh.} \\\\
    \\multicolumn{7}{l}{\\footnotesize **Based on 1.80 EUR/L.} \\\\
    \\multicolumn{7}{l}{\\footnotesize ***Based on 28 ct/kWh.} \\\\
""")

latex = latex.replace("lllllll", "lrrrrrr")

with open(tables_path / "stereo_vehicles.tex", "w+") as f:
    f.write(latex)

In [13]:
# Prepare figure comparing vehicle size trade-offs

# Small
df_small_icv = pd.DataFrame({
    "distance_km": np.linspace(0, 200),
    "cost_EUR": 210 / 22 + 9 * np.linspace(0, 200) * 1e-2,
})
df_small_icv["engine"] = "ICV"
df_small_icv["size"] = "Small"

df_small_bev = pd.DataFrame({
    "distance_km": np.linspace(0, 200),
    "cost_EUR": 260 / 22 + 4.48 * np.linspace(0, 200) * 1e-2,
})
df_small_bev["engine"] = "BEV"
df_small_bev["size"] = "Small"

# Medium
df_medium_icv = pd.DataFrame({
    "distance_km": np.linspace(0, 200),
    "cost_EUR": 260 / 22 + 10.80 * np.linspace(0, 200) * 1e-2,
})
df_medium_icv["engine"] = "ICV"
df_medium_icv["size"] = "Medium"

df_medium_bev = pd.DataFrame({
    "distance_km": np.linspace(0, 200),
    "cost_EUR": 400 / 22 + 5.60 * np.linspace(0, 200) * 1e-2,
})
df_medium_bev["engine"] = "BEV"
df_medium_bev["size"] = "Medium"

# Large
df_large_icv = pd.DataFrame({
    "distance_km": np.linspace(0, 200),
    "cost_EUR": 370 / 22 + 14.40 * np.linspace(0, 200) * 1e-2,
})
df_large_icv["engine"] = "ICV"
df_large_icv["size"] = "Large"

df_large_bev = pd.DataFrame({
    "distance_km": np.linspace(0, 200),
    "cost_EUR": 800 / 22 + 8.40 * np.linspace(0, 200) * 1e-2,
})
df_large_bev["engine"] = "BEV"
df_large_bev["size"] = "Large"

df = pd.concat([df_small_icv, df_small_bev, df_medium_icv, df_medium_bev, df_large_icv, df_large_bev])
df = df.rename(columns = { 
    "size": "Size", "engine": "Type",  
    "cost_EUR": "Daily cost [EUR]",
    "distance_km": "Daily distance [km]"
})

figure = px.line(df, x = "Daily distance [km]", y = "Daily cost [EUR]", color = "Type", facet_col = "Size")
figure.for_each_annotation(lambda item: item.update(text = item.text.split("=")[-1]))

figure.update_layout(
    width = 600,
    height = 200,
    margin=dict(
        l = 0,
        r = 0,
        b = 0,
        t = 15,
        pad = 0
    )
)

figure.write_image(figures_path / "vehicle_type_tradeoff.pdf")
figure